In [31]:
import pandas as pd

data = pd.read_csv('movie_dataset/train.csv')

data.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [32]:
data.info() #결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [33]:
#결측치 없음
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(data)

결측치가 존재하지 않습니다


In [34]:
#모델 설계를 위해 데이터를 문서와 label로 나눠줌

X = data.document #train 데이터에서 document 추출
Y = data.label # train 데이터에서 label(target) 추출

print(X.head())
print(Y.head())


0                  영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
1               히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...
2       괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~
3                아무래도 20년도지난작품이라 지금보기는너무유치하다
4    지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.
Name: document, dtype: object
0    0
1    1
2    1
3    0
4    0
Name: label, dtype: int64


In [35]:
#컴퓨터가 계산을 하기 위해선 문장을 숫자로 바꾼 다음 계산하도록 해야하는데 이것을 워드임베딩이라고 함
# 자연어 처리 분야에서 임베딩(Embedding)은 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자 형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미
#이 코드는 CountVectorizer 기법을 사용하여 문장을 숫자로 표현
#CounterVecotrizer는 입력된 문장을 토큰화(Tokenize)하여 토큰의 등장 빈도 벡터로 바꿔주는 기법
#토큰화란 문장을 의미가 있는 최소의 단위로 바꾸는 것
#이 코드에서는 띄어쓰기를 기준으로 토큰화 진행

from sklearn.feature_extraction.text import CountVectorizer #ConterVectorizer 사용을 위해 sklearn 패키지에 있는 CountVectorizer 불러옴


sample_vectorizer = CountVectorizer() #객체 생성


In [36]:
# CounterVectorizer를 학습시켜주어야 함
#어떠한 단어를 사용할지, 어떠한 단어가 중요도를 가지는지 학습하는 과정
#CounterVectorizer는 단어의 출현 빈도를 기준으로 문장을 벡터(=숫자의 집합)으로 바꿔줌 

sample_text1 = ["hello, my name is dacon and I am a data scientist!"]
sample_vectorizer.fit(sample_text1) #CountVectorizer 학습

CountVectorizer()

In [37]:
print(sample_vectorizer.vocabulary_) # Vocabulary는 단어 목록

#sklearn의 CountVectorizer는 "I", "a"와 같이 길이가 2 이하인 단어들과 ",",!" 같은 특수 문자는 의미가 적다고 판단하여 제외시킴
#최종적으로 위에 문장에서 9개의 단어가 Vocab에 포함됨


{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


In [38]:
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2) #sample_vectorizer를 활용해 벡터로 tramsform
print(sample_vector.toarray())

#단어의 출현 빈도로 이루어진 크기가 0인 벡터가 출력, 이 벡터를 BoW라고 함(CountVectorizer로 변환된 단어의 집합)
#sample_text2 문장 내에서는 전에 학습한 내용과 비교하여 동일한 단어인 dacon과 data 단어가 각 1번씩 출현함
#따라서 dacon과 data에 해당하는 인덱스는 값이 1로 표시

[[0 0 1 1 0 0 0 0 0]]


In [39]:
vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X) #train 데이터의 document를 countvectorizer로 학습시킴
X = vectorizer.transform(X) #벡터로 transform, X는 이제 자연어가 아닌 숫자, 벡터 데이터로 이루어져있음

#X는 머신러닝 모델의 input 값으로 적절한 데이터의 형식을 갖춤

In [40]:
#분류 문제에 많이 사용되는 LogisticRegression 모델로 분류
from sklearn.linear_model import LogisticRegression #모델 불러오기
model = LogisticRegression() #객체에 모델 할당
model.fit(X, Y) #모델 학습
#학습된 X값과 Y값을 머신러닝 모델에 넣어서 다시 학습

LogisticRegression()

In [41]:
#run model
#모델이 잘 학습되었는지 보기 위함
#꼭 예측값을 넣어야하는가? 그냥 모델에 넣고 돌려보면 안되나

X_pred = vectorizer.transform(["영화 완전 꿀잼"]) 
y_pred = model.predict(X_pred) #predict() 함수는 예측 결과 값
print(y_pred)
#0:negative 1:positive

[1]


In [43]:
test = pd.read_csv('movie_dataset/test.csv')
test.head()

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다


In [44]:
test_X = test.document

test_X_vect = vectorizer.transform(test_X) #document 데이터를 transform
#test 데이터를 대상으로 fit_transform 메소드를 실행하는 것은 test 데이털르 활용해 vectorizer를 학습시키는 것으로, data leakage에 해당
#즉, fit하지 않고 transform으로 변환시켜 모델에 학습

pred = model.predict(test_X_vect)
print(pred)

[0 1 0 ... 1 0 0]


In [45]:
submission = pd.read_csv('movie_dataset/sample_submission.csv')

submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [46]:
submission["label"] = pred #예측값 넣어주기
submission.head()

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1


In [47]:
submission.to_csv("submission1.csv", index=False)